# Auto-Encoder

Der Autoencoder versucht, eine Funktion $h_{W,b}(x) \approx x$ zu lernen. Mit anderen Worten, er versucht, eine Annäherung an die Identitätsfunktion zu lernen, um $\hat{x}$ auszugeben, die x ähnlich ist. Die Identitätsfunktion scheint eine besonders triviale Funktion zu sein, die zu lernen ist; aber indem wir dem Netzwerk Beschränkungen auferlegen, wie z.B. durch Begrenzung der Anzahl hidden layers, können wir eine interessante Struktur über die Daten entdecken. 

Als konkretes Beispiel nehmen wir an, die Eingaben x sind die Pixel-Intensitätswerte aus einem 28×28-Bild (784 Pixel), also n=784, und es gibt $s_2$=128 verborgene Einheiten in der Ebene $L_2$. Beachten Sie, dass wir auch $y \in \mathbb{R}^{784}$ haben. 

Da es nur 128 versteckte Einheiten gibt, ist das Netzwerk gezwungen, eine "komprimierte" Darstellung der Eingabe zu lernen. D.h., wenn es nur den Vektor der Aktivierungen der verborgenen Einheiten $a^{(2)} \in \mathbb{R}^{128}$ erhält, muss es versuchen, die 784-Pixel-Eingabe x zu "rekonstruieren". 

Wenn die Eingabe völlig zufällig wäre, wäre diese Komprimierungsaufgabe sehr schwierig. Wenn die Daten jedoch strukturiert sind, z.B. wenn einige der Eingabemerkmale korreliert sind, dann ist dieser Algorithmus in der Lage, einige dieser Korrelationen zu entdecken.

## Fashion MNIST Dataset

`Fashion-MNIST` ist ein Datensatz von Zalandos Artikelbildern - bestehend aus einem Trainingssatz mit 60.000 Beispielen und einem Testsatz mit 10.000 Beispielen. 

Jedes Beispiel ist ein Graustufenbild im Format 28x28, das mit einem Label aus 10 Klassen verbunden ist. Fashion-MNIST ist ein direkter Drop-in-Ersatz für den ursprünglichen MNIST-Datensatz von handgeschriebenen Ziffern. Der Datensatz hat die gleiche Bildgröße und die gleiche Struktur von Trainings- und Test-Splits.

Jedes Bild trägt einen der folgenden Label:

Label | Description
--- | ---
0|T-shirt/top
1|Trouser
2|Pullover
3|Dress
4|Coat
5|Sandal
6|Shirt
7|Sneaker
8|Bag
9|Ankle boot


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
# MNIST Dataset parameters.
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.01
training_steps = 20000
batch_size = 256
display_step = 1000

# Network Parameters
num_hidden_1 = 128 # 1st layer num features.
num_hidden_2 = 64 # 2nd layer num features (the latent dim).

### Daten laden

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

### Daten vorbereiten

Jedes Bild wird konvertiert zu float32, normalisiert auf das Intervall [0, 1] and ausgestreckt auf ein 1-dimensionales Array von 784 Features (28*28).

In [ ]:
# Convert to float32.
x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

Ein TensorFlow **`tf.data.Dataset`** dient der Verwaltung von potentiell gewaltigen Datenmengen.

Die `tf.data.Dataset` API unterstützt das Schreiben sprechender und effizienter Eingabe-Pipelines. Die Dataset-Verwendung folgt einem gemeinsamen Muster:

- Erstellen Sie einen Quelldatensatz aus Ihren Eingabedaten.
- Wenden Sie Dataset-Transformationen zur Vorverarbeitung der Daten an.
- Iterieren Sie über das Dataset und verarbeiten Sie die Elemente.

Die Iteration erfolgt in einem Streaming-Verfahren, so dass der vollständige Datensatz nicht in den Speicher passen muss.

**Einige Methoden:**
- `repeat(count)`: Wiederholt diesen Datensatz, so dass jeder Originalwert `count`-mal gesehen wird. Default: unendlich
- `shuffle(buffer_size)`: Mischt die Elemente dieses Datensatzes nach dem Zufallsprinzip um.

    Dieser Datensatz füllt einen Puffer mit buffer_size-Elementen, nimmt dann nach dem Zufallsprinzip Stichproben von Elementen aus diesem Puffer und ersetzt die ausgewählten Elemente durch neue Elemente. Für ein perfektes Mischen ist eine Puffergröße erforderlich, die größer oder gleich der vollen Größe des Datensatzes ist.

    Wenn Ihr Datensatz beispielsweise 10.000 Elemente enthält, buffer_size jedoch auf 1.000 eingestellt ist, wählt die Zufallsmischung zunächst ein Zufallselement aus nur den ersten 1.000 Elementen im Puffer aus. Sobald ein Element ausgewählt ist, wird sein Platz im Puffer durch das nächste (d. h. das 1.001-st.) Element ersetzt, wobei der Puffer mit 1.000 Elementen beibehalten wird.

- `batch(batch_size)`: Sammelt aufeinander folgende Elemente dieses Datensatzes zu Batch-Stapeln.
- `prefetch(buffer_size)`: Erstellt ein Dataset, das Elemente aus diesem Dataset vorzeitig abruft.

    Die meisten Dataset-Eingabe-Pipelines sollten mit einem Aufruf von `prefetch` enden. Auf diese Weise können spätere Elemente vorbereitet werden, während das aktuelle Element verarbeitet wird. Dies verbessert oft die Latenzzeit und den Durchsatz auf Kosten der Verwendung von zusätzlichem Speicher zum Speichern vorgeladener Elemente.

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(batch_size).prefetch(1)

test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.repeat().batch(batch_size).prefetch(1)


D.h. 

### Store layers weight & bias


In [ ]:
# A random value generator to initialize weights.
random_normal = tf.initializers.RandomNormal()

weights = {
    'encoder_h1': tf.Variable(random_normal([num_features, num_hidden_1])),
    'encoder_h2': tf.Variable(random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(random_normal([num_hidden_1, num_features])),
}
biases = {
    'encoder_b1': tf.Variable(random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(random_normal([num_features])),
}

### Building the encoder.

In [ ]:
def encoder(x):
    # Encoder Hidden layer with sigmoid activation.
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    
    # Encoder Hidden layer with sigmoid activation.
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    
    return layer_2

### Building the decoder

In [ ]:
def decoder(x):
    # Decoder Hidden layer with sigmoid activation.
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    
    # Decoder Hidden layer with sigmoid activation.
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    
    return layer_2

### Mean square loss between original images and reconstructed ones

In [ ]:
def mean_square(reconstructed, original):
    return tf.reduce_mean(tf.pow(original - reconstructed, 2))

### Adam optimizer

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

### Optimization process

In [ ]:
def run_optimization(x):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        reconstructed_image = decoder(encoder(x))
        loss = mean_square(reconstructed_image, x)

    # Variables to update, i.e. trainable variables.
    trainable_variables = {**weights, **biases}.values()
    
    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    
    return loss

### Trainiere das Modell

`take(count)` holt sich die nächsten `count` Batch-Elemente aus dem Dataset-Strom.


In [ ]:
for step, (batch_x, _) in enumerate(train_data.take(training_steps + 1)):
    
    # Run the optimization.
    loss = run_optimization(batch_x)
    
    if step % display_step == 0:
        print("step: %i, loss: %f" % (step, loss))

## Testing and Visualization

In [ ]:
import matplotlib.pyplot as plt

# Encode and decode images from test set and visualize their reconstruction.
n = 4

canvas_orig = np.empty((28 * n, 28 * n))
canvas_recon = np.empty((28 * n, 28 * n))

for i, (batch_x, _) in enumerate(test_data.take(n)):
    # Encode and decode the digit image.
    reconstructed_images = decoder(encoder(batch_x))

    # Display original images.
    for j in range(n):
        # Draw the generated digits.
        img = batch_x[j].numpy().reshape([28, 28])
        canvas_orig[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = img
    
    # Display reconstructed images.
    for j in range(n):
        # Draw the generated digits.
        reconstr_img = reconstructed_images[j].numpy().reshape([28, 28])
        canvas_recon[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = reconstr_img

print("Original Images")     
plt.figure(figsize=(n, n))
plt.imshow(canvas_orig, origin="upper", cmap="gray")
plt.show()

print("Reconstructed Images")
plt.figure(figsize=(n, n))
plt.imshow(canvas_recon, origin="upper", cmap="gray")
plt.show()